In [1]:
import csv
from bs4 import BeautifulSoup

In [2]:
from selenium import webdriver

In [3]:
driver = webdriver.Chrome()

In [4]:
url = 'https://www.amazon.com'
driver.get(url)

In [12]:
def get_url(search_term):
    template = "https://www.amazon.com/s?k={}&ref=nb_sb_noss"
        
    search_term = search_term.replace(' ', '+')
    return template.format(search_term)

In [14]:
url = get_url('ultrawide monitor')
print(url)

https://www.amazon.com/s?k=ultrawide+monitor&ref=nb_sb_noss


## extract the collection

In [16]:
soup = BeautifulSoup(driver.page_source, 'html.parser')

In [45]:
results = soup.find_all('div', {'data-component-type': 's-search-result'})
len(results)

21

## Prototype the record 

In [46]:
item = results[0]

In [47]:
atag = item.h2.a

In [52]:
description = atag.text.strip()

In [50]:
url = "https://www.amazon.com" + atag.get('href')


In [33]:
price_parent = item.find('span', 'a-price')

In [37]:
price = price_parent.find('span', 'a-offscreen').text

$226.36


In [40]:
rating = item.i.text

In [43]:
review_count = item.find('span', {'class': 'a-size-base', 'dir': 'auto'}).text

## Generalise the pattern

In [53]:
def extract_record(item):
    
    #description and url
    atag = item.h2.a
    description = atag.text.strip()
    url = "https://www.amazon.com" + atag.get('href')
    
    # price
    price_parent = item.find('span', 'a-price')
    price = price_parent.find('span', 'a-offscreen').text
    
    # rank and rating
    rating = item.i.text
    review_count = item.find('span', {'class': 'a-size-base', 'dir': 'auto'}).text
    
    asli_result = (description, price, rating, review_count,url)
    
    return asli_result
    

    

In [55]:
records = []
results = soup.find_all('div', {'data-component-type': 's-search-result'})

for item in results:
    records.append(extract_record(item))

AttributeError: 'NoneType' object has no attribute 'find'

## error handling

In [ ]:
def extract_record(item):
    
    #description and url
    atag = item.h2.a
    description = atag.text.strip()
    url = "https://www.amazon.com" + atag.get('href')
    
    try:
        
        # price
        price_parent = item.find('span', 'a-price')
        price = price_parent.find('span', 'a-offscreen').text
    except AttributeError:
        return
        
    try:
        
        # rank and rating
        rating = item.i.text
        review_count = item.find('span', {'class': 'a-size-base', 'dir': 'auto'}).text
    except AttributeError:
        rating = ''
        review_count = ''
    asli_result = (description, price, rating, review_count,url)
    
    return asli_result

In [63]:
records = []
results = soup.find_all('div', {'data-component-type': 's-search-result'})

for item in results:
    record = extract_record(item)
    if record:
        records.append(record)

AttributeError: 'NoneType' object has no attribute 'find'

In [64]:
records[0]


('LG 29WN600-W 29" 21:9 UltraWide WFHD IPS HDR10 Monitor with FreeSync, Silver',
 '$226.36',
 '4.6 out of 5 stars',
 '1,369',
 'https://www.amazon.com/LG-29WN600-W-29-21-UltraWide/dp/B0876DBCBX/ref=sr_1_1?dchild=1&keywords=ultrawide+monitor&qid=1616100781&sr=8-1')

## Getting the next page

In [65]:
def get_url(search_term):
    template = "https://www.amazon.com/s?k={}&ref=nb_sb_noss"
        
    search_term = search_term.replace(' ', '+')
    
    # add term query to url
    url = template.format(search_term)
    
    # add page query placeholder
    url += "&page{}"
    return url

## All together

In [69]:
import csv
from bs4 import BeautifulSoup
from selenium import webdriver

def get_url(search_term):
    template = "https://www.amazon.com/s?k={}&ref=nb_sb_noss"
        
    search_term = search_term.replace(' ', '+')
    
    # add term query to url
    url = template.format(search_term)
    
    # add page query placeholder
    url += "&page{}"
    return url

def extract_record(item):
    
    #description and url
    atag = item.h2.a
    description = atag.text.strip()
    url = "https://www.amazon.com" + atag.get('href')
    
    try:
        
        # price
        price_parent = item.find('span', 'a-price')
        price = price_parent.find('span', 'a-offscreen').text
    except AttributeError:
        return
        
    try:
        
        # rank and rating
        rating = item.i.text
        review_count = item.find('span', {'class': 'a-size-base', 'dir': 'auto'}).text
    except AttributeError:
        rating = ''
        review_count = ''
    asli_result = (description, price, rating, review_count,url)
    
    return asli_result


def main(search_term):
    driver = webdriver.Chrome()
    url = 'https://www.amazon.com'
    driver.get(url)
    
    record = []
    url = get_url(search_term)
    
    for page in range(1 , 21):
        driver.get(url.format(page))
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        results = soup.find_all('div', {'data-component-type': 's-search-result'})
        
        
        for item in results:
            record = extract_record(item)
            if record:
                records.append(record)
    
    driver.close()
    
    with open('results.csv', 'w', newline = '', encoding= 'utf-8') as f:
        writer = csv.writer(f)
        writer.writerow(['description', 'price', 'reviewcount','url' ])
        writer.writerows(records)
    


In [70]:
main('ultrawide monitor')